## Langgraph Basics

![lang-graph](images/langgraph.jpg)

What is LangGraph?

LangGraph is a library built on top of LangChain, designed to add cyclic computational capabilities to your LLM applications. While LangChain allows you to define chains of computation (Directed Acyclic Graphs or DAGs), LangGraph introduces the ability to add cycles, enabling more complex, agent-like behaviors where you can call an LLM in a loop, asking it what action to take next.

* Stateful Graph: LangGraph revolves around the concept of a stateful graph, where each node in the graph represents a step in your computation, and the graph maintains a state that is passed around and updated as the computation progresses.

* Nodes: Nodes are the building blocks of your LangGraph. Each node represents a function or a computation step. You define nodes to perform specific tasks, such as processing input, making decisions, or interacting with external APIs.

* Edges: Edges connect the nodes in your graph, defining the flow of computation. LangGraph supports conditional edges, allowing you to dynamically determine the next node to execute based on the current state of the graph


Source: https://medium.com/@cplog/introduction-to-langgraph-a-beginners-guide-14f9be027141


In LangGraph, both MessageGraph and StateGraph are used to represent and manage the state of a graph during its execution. However, they differ in how they structure and update this state.

### MessageGraph:

* State Structure: The state is a simple list of messages.
* Update Mechanism: Each node's output (typically a message) is appended to the state list after execution.
* Ideal Use Case: Best suited for applications involving conversational AI or chatbots, where the state primarily consists of a history of messages exchanged between the user and the AI.

### StateGraph:

* State Structure: The state is a more general dictionary (or object), where different keys can store different types of information relevant to the graph's execution.
* Update Mechanism: Each node can update any part of the state dictionary based on its output and logic.
* Ideal Use Case: Offers greater flexibility, making it suitable for a wider range of applications where the state needs to store more complex information beyond just messages, such as user preferences, environment variables, or intermediate results.

#### Key Difference:

The main difference lies in the structure and flexibility of the state. MessageGraph is a specialized version of StateGraph, optimized for scenarios where the state is primarily a conversation history. StateGraph provides more flexibility for storing and updating arbitrary state information.

#### Basic Langgraph example

In [8]:
from langgraph.graph import Graph

def greet(input):
    return "Hello "

def user(input):
    return input + "Sam!"
    
#Create a graph

workflow = Graph()

#Add python functions as nodes
workflow.add_node("greet_node", greet)
workflow.add_node("user_node", user)

#Add an edge
workflow.add_edge("greet_node", "user_node")

workflow.set_entry_point("greet_node")
workflow.set_finish_point("user_node")

app = workflow.compile()
app.get_graph().print_ascii()


+-----------+  
| __start__ |  
+-----------+  
       *       
       *       
       *       
+------------+ 
| greet_node | 
+------------+ 
       *       
       *       
       *       
+-----------+  
| user_node |  
+-----------+  
       *       
       *       
       *       
  +---------+  
  | __end__ |  
  +---------+  


In [9]:
input = 'Hi'
for output in app.stream(input):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'greet_node':
---
Hello 

---

Output from node 'user_node':
---
Hello Sam!

---



### Langggraph example with LLM Model

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") ## Put your OpenAI API key here
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_HUB_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_TRACING_V2"] = 'true' ## Set this as True
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com/' ## Set this as: https://api.smith.langchain.com/
os.environ["LANGCHAIN_PROJECT"] = 'llm-langgraph'

In [12]:
from langgraph.graph import Graph
from langchain_openai import ChatOpenAI

# Set the model as ChatOpenAI
model = ChatOpenAI(temperature=0) 

def joker(input):
    prompt = f"Tell me a joke about {input}"
    response = model.invoke(prompt)
    return response.content

def critic(input):
    prompt = f"Rate and Criticize this joke: {input}"
    response = model.invoke(prompt)
    return response.content
    
#Create a graph

workflow = Graph()

workflow.add_node("Create_Joke", joker)
workflow.add_node("Criticize_Joke", critic)

workflow.add_edge("Create_Joke", "Criticize_Joke")

workflow.set_entry_point("Create_Joke")
workflow.set_finish_point("Criticize_Joke")

app = workflow.compile()
app.get_graph().print_ascii()


  +-----------+    
  | __start__ |    
  +-----------+    
         *         
         *         
         *         
  +-------------+  
  | Create_Joke |  
  +-------------+  
         *         
         *         
         *         
+----------------+ 
| Criticize_Joke | 
+----------------+ 
         *         
         *         
         *         
    +---------+    
    | __end__ |    
    +---------+    


In [13]:
input = 'Canadians'
for output in app.stream(input):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'Create_Joke':
---
Why did the Canadian cross the road?

To get to the Tim Hortons on the other side, eh!

---

Output from node 'Criticize_Joke':
---
I would rate this joke a 6 out of 10. It plays on the stereotype of Canadians loving Tim Hortons, which can be funny to some people. However, it is a bit predictable and not very original. Adding the "eh" at the end is a nice touch to emphasize the Canadian stereotype, but overall the joke could use a bit more creativity.

---

